# Context-Enriched Retrieval in RAG
RAG(Retrieval-Augmented Generation, 검색 기반 생성)은 외부 지식에서 관련 정보를 찾아와 AI의 응답 품질을 높이는 방식입니다. 하지만 기존의 검색 방식은 서로 떨어진 텍스트 조각들만 가져오기 때문에, 답변이 불완전하거나 맥락이 끊길 수 있습니다.

이 문제를 해결하기 위해, 문맥을 강화한 검색(Context-Enriched Retrieval) 방식을 소개합니다. 이 방법은 관련된 정보뿐만 아니라 주변 내용까지 함께 가져와 보다 자연스럽고 완성도 높은 답변을 가능하게 합니다.

이 섹션에서는 다음과 같은 단계를 거칩니다.
- 데이터 불러오기: PDF 파일에서 텍스트를 추출합니다.
- 겹치는 구간으로 나누기: 문맥을 유지하기 위해 텍스트를 겹치는 조각들로 나눕니다.
- 임베딩 생성: 텍스트 조각들을 숫자 벡터로 변환합니다.
- 문맥 기반 검색: 관련된 텍스트와 그 주변 내용을 함께 검색합니다.
- 응답 생성: 검색된 문맥을 바탕으로 언어 모델이 답변을 생성합니다.
- 결과 평가: 생성된 답변의 정확도를 평가합니다.

## 환경 설정하기
필요한 라이브러리를 가져오는 것부터 시작합니다.

In [54]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)


## PDF 파일에서 텍스트 추출하기

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [3]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

2023 핵심 개정세법
01
2024 달라지는 세금제도
(국민·기업 납세자용)

[유의사항]
'2023 핵심 개정세법'은 국회에서 의결된 세법 개정사항을 모두 포괄하고 있으나, 시행령·
시행규칙의 경우 2023.7월 발표한 ‘2023 세법개정안'을 중심으로 개정세법과 관련된 내용의
경우 그대로 반영하였습니다. 그러므로 정부의 시행령, 시행규칙 개정 과정에서 일부 변동될
수 있고 새로이 추가 제정될 수도 있습니다. 아울러 실무상 적용할 때는 반드시 개정세법의
구체적인 조문을 확인하셔야 합니다.

국민·기업 납세자용
2024 달라지는 세금제도
2024 달라지는 세금제도
2023 세목별 핵심 개정세법
2023 개정세법 현행-개정사항 비교
01
부동산 세금제도
●(조특법) 연 단위 양도세 감면한도 악용방지를 위해 감면한도 산정방법 조정
양도소득세 산정 및 감면이 연단위로 이뤄지는 점을 감안하여 ▲토지의 일부를
양도한 날부터 소급하여 1년 내 토지를 분할한 경우 분필한 토지 또는 토지 지분



## 추출된 텍스트의 chunkibng
추출된 텍스트를 얻은 후, 검색 정확성을 향상시키기 위해 이를 더 작고 겹치는 청크로 나눕니다.

In [8]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

## Extracting and Chunking Text from a PDF File
Now, we load the PDF, extract text, and split it into chunks.

In [9]:
# 추출된 텍스트를 1000자 크기의 청크로 나눔(overlap 200)
text_chunks = chunk_text(extracted_text, 1000, 200)

# 청크 개수 출력
print("Number of text chunks:", len(text_chunks))

# 첫 번째 청크 출력
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 35

First text chunk:
2023 핵심 개정세법
01
2024 달라지는 세금제도
(국민·기업 납세자용)

[유의사항]
'2023 핵심 개정세법'은 국회에서 의결된 세법 개정사항을 모두 포괄하고 있으나, 시행령·
시행규칙의 경우 2023.7월 발표한 ‘2023 세법개정안'을 중심으로 개정세법과 관련된 내용의
경우 그대로 반영하였습니다. 그러므로 정부의 시행령, 시행규칙 개정 과정에서 일부 변동될
수 있고 새로이 추가 제정될 수도 있습니다. 아울러 실무상 적용할 때는 반드시 개정세법의
구체적인 조문을 확인하셔야 합니다.

국민·기업 납세자용
2024 달라지는 세금제도
2024 달라지는 세금제도
2023 세목별 핵심 개정세법
2023 개정세법 현행-개정사항 비교
01
부동산 세금제도
●(조특법) 연 단위 양도세 감면한도 악용방지를 위해 감면한도 산정방법 조정
양도소득세 산정 및 감면이 연단위로 이뤄지는 점을 감안하여 ▲토지의 일부를
양도한 날부터 소급하여 1년 내 토지를 분할한 경우 분필한 토지 또는 토지 지분
의 일부를 양도하거나 ▲ 토지(또는 지분) 일부를 양도하고 2년 내 나머지 토지를
동일인이나 배우자에게 양도한 경우 1개 과세기간 내 양도한 것으로 보고 양도소
득세 종합한도를 적용하도록 하여 조세회피를 막도록 했다.
(2024.1.1. 이후 양도하는 분부터 적용)
●(조특법) '기회발전특구'에 있는 주택 추가 취득시 1세대 1주택 양도세 비과세
적용
기회발전특구를 농어촌주택 특례 소재지에 포함하여 특구 내에서 주택을 추가로
취득하여도 특구 내 주택 외 일반주택을 양도 시에는 특구 내 주택은 없는 것으로
보고 1세대 1주택 양도소득세 비과세 특례 적용하는 혜택을 받을 수 있게 되었다.
(2024.1.1. 이후 양도하는 분부터 적용)
3

한국세무사회
02
주식 세금제도
●(소득세법) 배당소득 이중과세 조정을 위한 배당가산율 조정
개인이 기업 등으로부터 배당금을 받아 종합과세되는 경우 배당소득 이중과세
조

## 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [10]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

Using device: mps


In [34]:
# 배치 크기를 줄여 임베딩을 생성합니다.
embeddings = create_embeddings(embedding_model, text_chunks, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Generated 35 sentence embeddings.


## Context-Aware Semantic Search 구현
chunk와 인접한 청크를 포함하도록 수정


In [35]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    가장 관련성이 높은 청크와 그 이웃 청크를 검색합니다.

    Args:
    query (str): 검색 쿼리.
    text_chunks (List[str]): 텍스트 청크의 리스트.
    embeddings (List[dict]): 청크 임베딩의 리스트.
    k (int): 검색할 관련 청크의 수.
    context_size (int): 포함할 이웃 청크의 수.

    Returns:
    List[str]: 문맥 정보를 포함한 관련 텍스트 청크.
    """
    # 쿼리를 임베딩 벡터로 변환
    query_embedding =  create_embeddings(embedding_model, [query], device=device)[0]
    similarity_scores = []

    # 쿼리와 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    for i, chunk_embedding in enumerate(embeddings):
        # 쿼리 임베딩과 현재 청크 임베딩 간의 코사인 유사도 계산
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        # 인덱스와 유사도 점수를 튜플로 저장
        similarity_scores.append((i, similarity_score))


    # 유사도 점수에 따라 청크를 내림차순으로 정렬 (가장 높은 유사도 우선)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 가장 관련성이 높은 청크의 인덱스 가져오기
    top_index = similarity_scores[0][0]

    # 문맥 포함 범위 정의
    # 0보다 작거나 text_chunks의 길이를 초과하지 않도록 보장
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 관련 청크와 그 이웃 문맥 청크 반환
    return [text_chunks[i] for i in range(start, end)]

## 쿼리로 context 검색

In [37]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

,query,generation_gt
0,피고의 고지의무 위반으로 인해 원고들이 입은 손해는 무엇으로 정의됩니까?,"피고의 고지의무 위반으로 인해 원고들이 입은 손해는 ""이 사건 각 분양계약에 따른 ..."
1,지방공무원법 제53조는 공무원이 직무와 관련하여 무엇을 주거나 받을 수 없다고 규정...,"지방공무원법 제53조는 공무원이 직무와 관련하여 직접적이든 간접적이든 사례, 증여 ..."
2,연금소득의 분리과세 기준금액은 얼마로 인상되었습니까?,연금소득의 분리과세 기준금액은 1천200만원에서 1천500만원으로 인상되었습니다.
3,공무원연금법 제65조 제1항 제1호에 의한 퇴직연금 등의 감액 요건은 무엇입니까?,공무원연금법 제65조 제1항 제1호에 의한 퇴직연금 등의 감액 요건은 재직 중 직무...
4,권리사용료 산출방법에 대한 고시는 언제 개정되었습니까?,권리사용료 산출방법에 대한 고시는 2021년 3월 30일에 개정되었습니다.


In [38]:
# 평가 데이터에서 첫 번째 쿼리를 추출합니다.
query = df['query'][0]

# 가장 관련성이 높은 청크와 그 이웃 청크를 검색하여 문맥 제공
# 매개변수:
# - query: 검색할 질문
# - text_chunks: PDF에서 추출한 텍스트 청크
# - response.data: 텍스트 청크의 임베딩
# - k=1: 가장 일치하는 항목 반환
# - context_size=1: 문맥을 위해 가장 일치하는 항목 전후로 1개의 청크 포함
top_chunks = context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1)

# 참조를 위해 쿼리 출력
print("쿼리:", query)
# 각 검색된 청크를 제목과 구분선과 함께 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n=====================================")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

14
---
13 16
쿼리: 피고의 고지의무 위반으로 인해 원고들이 입은 손해는 무엇으로 정의됩니까?
문맥 1:
은 세율을 적용하였으나 120억원까지로 대폭 확대되었다. 다만 가
업승계 목적의 증여 시 그 증여자 또는 수증자가 기업의 경영과 관련한 조세포탈
또는 회계부정 행위로 처벌(증여일 10년전 ~ 증여후 5년 이내)을 받은 경우에는 과
세특례 적용이 배제된다.
(2024.1.1. 이후 증여분부터 적용)
2023 핵심 개정세법
14

국민·기업 납세자용
2024 달라지는 세금제도
2023 세목별 핵심 개정세법
2023 개정세법 현행-개정사항 비교
06 기업경영 세금제도
● (소득세법, 법인세법) 상용근로소득에 대한 간이지급명세서 2024년부터 매월 제
출제도가 현재처럼 반기제출 유지(2년 유예) [구재이 회장 공약, 한국세무사회
입법안]
▲상용근로자에 대한 근로소득 간이지급명세서를 2024년 1월 1일부터 매월 제
출하도록 작년 말 개정하였으나, 「소득기반 고용보험」시행 지연에 따라 시행시기
를 2026년 1월 1일로 2년 유예했다. 이에 따라 상용근로소득 간이지급명세서는
현재처럼 반기제출을 하면 된다. ▲상용근로소득 간이지급명세서 매월 제출시행이
유예됨에 따라 지연제출가산세, 세액공제 신설규정의 시행도 2년 유예되었다.
● (소득세법) 3.3% 사업소득은 소액부징수 대상에서 제외
사업소득 원천징수 합리화를 위해 계속적, 반복적으로 행하는 활동을 통하여 얻
는 인적용역 사업소득은 소액이더라도 예외 없이 원천징수하도록 하였다.
(2024.7.1. 이후 지급하는 분부터 적용)
● (소득세법, 법인세법) '외국인 통합계좌'에 의한 명의인 원천징수 제도 신설
지급받는 개개의 외국인에게 원천징수를 하여야 하나, ‘외국인 통합계좌'(외국
금융투자업자가 다른 외국투자자의 주식매매거래를 일괄해 주문·결제하기 위해 자
기명의로 개설한 계좌)를 통해 투자시 소득지급자는 통합계좌 명의인에 대해 원천
징수(조세조약에 따른 비과세, 면제, 제한세율 미적용)하며, 원천징수 이후 

## Generating a Response Using Retrieved Context
We now generate a response using LLM.

In [57]:


def generate_response(system_prompt, user_message ,model_name='gpt-4.1-mini'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 어시스턴트의 시스템 프롬프트를 정의합니다.
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt,'gpt-4.1-nano-2025-04-14')

In [58]:
print(ai_response.choices[0].message.content)

해당 질문에 답변할 충분한 정보가 없습니다. 제공된 컨텍스트에는 피고의 고지의무 위반으로 인해 원고들이 입은 손해에 대한 구체적인 내용이 포함되어 있지 않습니다.


## 생성 응답 평가하기
생서 응답을 예상 답변과 비교하여 점수를 부여합니다.

In [59]:
# 평가 시스템의 시스템 프롬프트를 정의합니다.
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 사용자 쿼리, AI 응답, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답을 생성합니다.
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

# 평가 응답을 출력합니다.
print(evaluation_response.choices[0].message.content)

0


# 전체 추론 평가

In [ ]:
# 생성된 답변과 실제 답변을 비교하여 점수를 부여합니다.
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1)

    # 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
    user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
    user_prompt = f"{user_prompt}\n질문: {query}"

    # AI 어시스턴트의 시스템 프롬프트를 정의합니다.
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

    # response 생성
    ai_response = generate_response(system_prompt, user_prompt, 'gpt-4.1-mini')

    print(ai_response.choices[0].message.content)

    # 평가 프롬프트를 정의합니다.
    evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5. \n The answer should only output numbers and should not output any words."

    # 사용자 쿼리, AI response, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][i]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

    # 평가 응답을 출력합니다.
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)

In [61]:
import numpy as np
result = [float(x.replace('\n','').replace('Score: ',''))for x in result]


In [62]:
np.average(result)

0.5666666666666667